In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import lightgbm as lgb
import torch
import torch.nn as nn
import torch.optim as optim
import pickle

In [2]:
def resample_and_interpolate(group):
    return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()

In [3]:
weather_df = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/weather_data/DWD_ICON-EU.csv')
solar_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv')
wind_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/wind_total_production.csv')
solar_total.generation_mw = solar_total.generation_mw * 0.5
wind_total.generation_mw = wind_total.generation_mw * 0.5 - wind_total.boa
weather_df.sort_values(by='ref_datetime', inplace=True)
weather_df = weather_df.groupby(["valid_datetime","latitude","longitude"]).last().reset_index()
weather_df.reset_index(inplace=True)

In [4]:
weather_df.valid_datetime = pd.to_datetime(weather_df.valid_datetime)
weather_df = weather_df.set_index(["valid_datetime","latitude","longitude"])
df_resampled = weather_df.groupby(['latitude', 'longitude'], group_keys=False).apply(resample_and_interpolate)
df_resampled = df_resampled.reset_index()

C:\Users\paulh\AppData\Local\Temp\ipykernel_28808\2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
C:\Users\paulh\AppData\Local\Temp\ipykernel_28808\2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
C:\Users\paulh\AppData\Local\Temp\ipykernel_28808\2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
C:\Users\paulh\AppData\Local\Temp\ipykernel_28808

In [5]:
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
wind_total.timestamp_utc = pd.to_datetime(wind_total.timestamp_utc)
df_resampled.drop(columns=['index','ref_datetime'], inplace=True)
df_resampled_merged = pd.merge(df_resampled, solar_total, how='left', left_on='valid_datetime', right_on='timestamp_utc')
df_resampled_merged_solar = df_resampled_merged.loc[~(df_resampled_merged.latitude == 53.935) & ~(df_resampled_merged.longitude == 1.8645)]
df_resampled_merged_solar1 = df_resampled_merged_solar.groupby("valid_datetime").mean().reset_index()
distinct_lat_lon_pairs = df_resampled_merged_solar[['latitude', 'longitude']].drop_duplicates()

In [6]:
df_resampled_merged_wind = pd.merge(df_resampled, wind_total, how='inner', left_on='valid_datetime', right_on='timestamp_utc')
df_resampled_merged_wind = df_resampled_merged_wind.loc[(df_resampled_merged_wind.latitude == 53.935) & (df_resampled_merged_wind.longitude == 1.8645)]
df_resampled_merged_wind.drop_duplicates(inplace=True)
df_resampled_merged_wind


,valid_datetime,latitude,longitude,Temperature,WindSpeed,WindSpeed:100,WindDirection:100,CloudCover,RelativeHumidity,PressureReducedMSL,SolarDownwardRadiation,TotalPrecipitation,timestamp_utc,settlement_date,settlement_period,boa,generation_mw
20,2024-10-07 00:00:00+00:00,53.935,1.8645,14.930556,8.763889,11.313333,159.952778,1.000000,93.232778,99385.651667,0.0,NaN,2024-10-07 00:00:00+00:00,2024-10-07,3,0.0,482.321
41,2024-10-07 00:30:00+00:00,53.935,1.8645,14.891944,9.500139,12.232361,162.957083,1.000000,94.189028,99358.019583,0.0,NaN,2024-10-07 00:30:00+00:00,2024-10-07,4,0.0,541.471
62,2024-10-07 01:00:00+00:00,53.935,1.8645,14.853333,10.236389,13.151389,165.961389,1.000000,95.145278,99330.387500,0.0,1.614722,2024-10-07 01:00:00+00:00,2024-10-07,5,0.0,569.911
83,2024-10-07 01:30:00+00:00,53.935,1.8645,14.865694,9.685278,12.327083,184.986806,1.000000,94.389861,99286.175278,0.0,0.876389,2024-10-07 01:30:00+00:00,2024-10-07,6,0.0,568.161
104,2024-10-07 02:00:00+00:00,53.935,1.8645,14.878056,9.134167,11.502778,204.012222,1.000000,93.634444,99241.963056,0.0,0.138056,2024-10-07 02:00:00+00:00,2024-10-07,7,0.0,568.211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926,2024-10-10 19:30:00+00:00,53.935,1.8645,9.262917,10.524971,11.384034,325.659167,0.820139,77.654722,100965.528611,0.0,0.055694,2024-10-10 19:30:00+00:00,2024-10-10,42,0.0,361.670
3947,2024-10-10 20:00:00+00:00,53.935,1.8645,9.337222,10.478575,11.341824,325.081111,0.787500,76.886111,101009.741111,0.0,0.015556,2024-10-10 20:00:00+00:00,2024-10-10,43,0.0,410.810
3968,2024-10-10 20:30:00+00:00,53.935,1.8645,9.492500,10.386034,11.240244,323.263472,0.655000,74.927083,101017.684167,0.0,0.010694,2024-10-10 20:30:00+00:00,2024-10-10,44,0.0,423.910
3989,2024-10-10 21:00:00+00:00,53.935,1.8645,9.647778,10.293493,11.138664,321.445833,0.522500,72.968056,101025.627222,0.0,0.005833,2024-10-10 21:00:00+00:00,2024-10-10,45,0.0,417.860


In [7]:
def set_up_wind_features(df):
    R_d = 287.05  # Specific gas constant for dry air (J/(kg·K))
    R_v = 461.5   # Specific gas constant for water vapor (J/(kg·K))
    p = 101325    # Standard atmospheric pressure in Pa
    # Calculate saturation vapor pressure (using temperature in Celsius), Tetens formula
    df['Temperature_K'] = df['Temperature'] + 273.15
    e_s = 0.61078 * np.exp((17.27 * (df['Temperature'])) / (df['Temperature'] +237.3))
    # in pa
    e_s = 1000 * e_s
    # Calculate actual vapor pressure
    e = df['RelativeHumidity'] / 100 * e_s
    df['AirDensity'] = (p - e) / (R_d * df['Temperature_K']) + (e / (R_v * df['Temperature_K']))
    # Turbine stats
    rotor_diameter = 154  # in meters
    approximated_total_efficiency = 0.348
    limiter = 0.94
    minimum_wind_speed = 3  # in m/s
    maximum_wind_speed_for_power_curve = 12.5  # in m/s
    maximum_wind_speed_for_operation = 25  # in m/s
    rotor_area = np.pi * (rotor_diameter / 2) ** 2  # in m²
    # turbine requires 3m/s to start rotating
    const_internal_friction_coefficient = 0.5 * 1.240 * np.pi * 77**2 * 3**3 * approximated_total_efficiency * 174 / 1000000
    maximum_power_per_turbine = 7 # in MW
    # Same for full
    df['WindSpeed_full_avg'] = (df['WindSpeed'] + df['WindSpeed:100']) / 2
    df['WindPower_full'] = 0.5 * df['AirDensity'] * rotor_area * df['WindSpeed:100'] ** 3 * 174 / 1000000
    df['UsableWindPower_full'] = np.minimum(df['WindPower_full'], maximum_power_per_turbine * 174 * limiter / approximated_total_efficiency)
    df['PowerOutput_full'] = np.where((df['WindSpeed:100'] >= minimum_wind_speed) & (df['WindSpeed:100'] <= maximum_wind_speed_for_operation), df['UsableWindPower_full'] * approximated_total_efficiency - const_internal_friction_coefficient, 0)

    # wind_df["Temperature_avg"] = (wind_df["Temperature"] + wind_df["Temperature:100"]) / 2
    # wind_df["RelativeHumidity_avg"] = (wind_df["RelativeHumidity"] + wind_df["RelativeHumidity:100"]) / 2
    df["Temperature_avg"] = df["Temperature"]
    df["RelativeHumidity_avg"] = df["RelativeHumidity"]   
    df["WindSpeed:100_dwd_lag1"] = df["WindSpeed:100"].shift(1)
    df["WindSpeed:100_dwd_lag2"] = df["WindSpeed:100"].shift(2)
    df["WindSpeed:100_dwd_lag3"] = df["WindSpeed:100"].shift(3)
    df["UsableWindPower_opt"] = df.UsableWindPower_full
    df["WindSpeed:100_dwd"] = df["WindSpeed:100"].shift(1)
    return df

In [8]:
df_resampled_merged_wind_2 = set_up_wind_features(df_resampled_merged_wind)
df_resampled_merged_wind_2.dropna(inplace=True)
X_wind = df_resampled_merged_wind_2[['WindSpeed:100_dwd', 'Temperature_avg', 'RelativeHumidity_avg', 'AirDensity',"UsableWindPower_opt", 'WindSpeed:100_dwd_lag1', 'WindSpeed:100_dwd_lag2', 'WindSpeed:100_dwd_lag3']]
y_wind = df_resampled_merged_wind_2['generation_mw']

In [9]:
def set_up_solar_features(df):
    df["hour"] = df.valid_datetime.dt.hour
    df["day_of_year"] = df.valid_datetime.dt.dayofyear
    df["cos_day_of_year"] = np.cos(2 * np.pi * df.day_of_year / 365)
    df["cos_hour"] = np.cos(2 * np.pi * df.hour / 24)
    df["Mean_SolarDownwardRadiation"] = df.SolarDownwardRadiation
    df["Mean_Temperature"] = df.Temperature
    df["Std_Temperature"] = df_resampled_merged_solar.groupby("valid_datetime").std().reset_index().Temperature
    df["SolarDownwardRadiation_RW_Mean_30min"] = df.Mean_SolarDownwardRadiation.rolling(window=1, min_periods=1).mean()
    df["SolarDownwardRadiation_RW_Mean_1hour"] = df.Mean_SolarDownwardRadiation.rolling(window=2, min_periods=1).mean()
    df["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df.Mean_SolarDownwardRadiation.shift(1)
    df["SolarDownwardRadiation_dwd_Mean_Lag_1h"] = df.Mean_SolarDownwardRadiation.shift(2)
    df["SolarDownwardRadiation_dwd_Mean_Lag_24h"] = df.Mean_SolarDownwardRadiation.shift(48)
    for i in range(len(distinct_lat_lon_pairs)):
        lat = distinct_lat_lon_pairs.latitude.iloc[i]
        lon = distinct_lat_lon_pairs.longitude.iloc[i]
        mask = (df_resampled_merged_solar.latitude == lat) & (df_resampled_merged_solar.longitude == lon)
        df[f"Temperature_{i}"] = pd.Series(df_resampled_merged_solar.Temperature[mask].values)[:len(df)]  # Fill gaps with NaN
        df[f"SolarDownwardRadiation_{i}"] = pd.Series(df_resampled_merged_solar.SolarDownwardRadiation[mask].values)[:len(df)]  # Fill gaps with NaN
    return df
df_resampled_merged_solar2 = set_up_solar_features(df_resampled_merged_solar1)

In [10]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    # Calculate cell temperature using the simplified NOCT model
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    
    # Calculate the efficiency loss due to increased cell temperature
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [11]:
for i in range(20):
    temp_col = f'Temperature_{i}'
    irradiance_col = f'SolarDownwardRadiation_{i}'
    panel_temp_col = f'Panel_Temperature_Point{i}'
    panel_eff_col = f'Panel_Efficiency_Point{i}'
    df_resampled_merged_solar2[panel_temp_col], df_resampled_merged_solar2[panel_eff_col] = pv_temperature_efficiency(df_resampled_merged_solar2[irradiance_col], df_resampled_merged_solar2[temp_col])
df_resampled_merged_solar2["Panel_Temperature_dwd_mean"] = df_resampled_merged_solar2.filter(regex= r"Panel_Temperature.*").mean(axis= 1)
df_resampled_merged_solar2["Panel_Efficiency_dwd_mean"] = df_resampled_merged_solar2.filter(regex= r"Panel_Efficiency.*").mean(axis= 1)
df_resampled_merged_solar2["Panel_Temperature_dwd_std"] = df_resampled_merged_solar2.filter(regex= r"Panel_Temperature.*").std(axis= 1)
df_resampled_merged_solar2["Panel_Efficiency_dwd_std"] = df_resampled_merged_solar2.filter(regex= r"Panel_Efficiency.*").std(axis= 1)
df_resampled_merged_solar2["solar_mw_lag_48h"] = df_resampled_merged_solar2.generation_mw.shift(periods= 96)
df_resampled_merged_solar2["capacity_mwp_lag_48h"] = df_resampled_merged_solar2.capacity_mwp.shift(periods= 96)
df_resampled_merged_solar2["Target_Capacity_MWP%"] = df_resampled_merged_solar2.generation_mw / df_resampled_merged_solar2.capacity_mwp
df_resampled_merged_solar2["Target_Capacity_MWP%_lag_48h"] = df_resampled_merged_solar2["Target_Capacity_MWP%"].shift(periods= 96)


C:\Users\paulh\AppData\Local\Temp\ipykernel_28808\3533751230.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_resampled_merged_solar2["Target_Capacity_MWP%_lag_48h"] = df_resampled_merged_solar2["Target_Capacity_MWP%"].shift(periods= 96)


In [12]:
df_resampled_merged_solar3 = df_resampled_merged_solar2[[ 
    "Mean_SolarDownwardRadiation",
    "SolarDownwardRadiation_RW_Mean_1hour",
    "SolarDownwardRadiation_RW_Mean_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_24h",
    "Panel_Efficiency_dwd_mean",
    "Panel_Efficiency_dwd_std",
    "Panel_Temperature_dwd_mean",
    "Panel_Temperature_dwd_std",
    "Std_Temperature",
    "Mean_Temperature",
    "cos_hour",
    "cos_day_of_year",
    "solar_mw_lag_48h",
    "capacity_mwp_lag_48h",
    "Target_Capacity_MWP%_lag_48h",
    "Target_Capacity_MWP%"
    ]]
df_resampled_merged_solar3.dropna(inplace=True)
Y_solar = df_resampled_merged_solar3["Target_Capacity_MWP%"]
X_solar = df_resampled_merged_solar3.drop(columns=["Target_Capacity_MWP%"])
X_solar.capacity_mwp_lag_48h.mean()

C:\Users\paulh\AppData\Local\Temp\ipykernel_28808\3836400376.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resampled_merged_solar3.dropna(inplace=True)


2779.280681259579

In [13]:
def modified_pinball_loss(y_true, y_pred, quantile):
    delta = y_true - y_pred
    return np.mean(np.maximum(quantile * delta, (quantile - 1) * delta))

In [14]:
from sklearn.ensemble import HistGradientBoostingRegressor  # Dies ist nur für die Typisierung notwendig

In [15]:
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.exceptions import NotFittedError

In [16]:
import joblib
def load_pickle1(path):
    return joblib.load(path)

In [17]:
df_resampled_merged_wind_2[['WindPower_full','UsableWindPower_full','PowerOutput_full']]

,WindPower_full,UsableWindPower_full,PowerOutput_full
83,3698.122715,3290.000000,1126.039423
104,3004.751994,3004.751994,1026.773117
125,2951.020062,2951.020062,1008.074404
146,2897.910196,2897.910196,989.592171
167,3374.051946,3290.000000,1126.039423
...,...,...,...
3926,2978.111145,2978.111145,1017.502101
3947,2944.381408,2944.381408,1005.764153
3968,2864.549308,2864.549308,977.982582
3989,2786.203975,2786.203975,950.718406


In [18]:
import plotly.express as px
import plotly.graph_objects as go
from sklearn.exceptions import InconsistentVersionWarning
import warnings

# Suppress the version mismatch warning
warnings.filterwarnings("ignore", category=InconsistentVersionWarning)
def modified_pinball_loss(y_true, y_pred, quantile):
    error = y_true - y_pred
    return np.mean(np.maximum(quantile * error, (quantile - 1) * error))

path = 'D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/Generation_forecast/Wind_forecast/models/gbr_quantile_0.'
quantiles = [1, 2, 3, 4, 5, 6, 7, 8, 9]
losses = []

for quantile in quantiles:
    with open(f"{path}{quantile}_res-True_calc-False.pkl", "rb") as f:
        model_wind = load_pickle1(f)
    
    if not hasattr(model_wind, '_preprocessor'):
        model_wind._preprocessor = None

    predictions_wind = model_wind.predict(X_wind)
    df_resampled_merged_wind_2[f"generation_mw_quantile_{quantile}"] = predictions_wind + df_resampled_merged_wind_2.PowerOutput_full /2
    loss = modified_pinball_loss(y_wind, df_resampled_merged_wind_2[f"generation_mw_quantile_{quantile}"], quantile/10)
    losses.append(loss)
    print(f"Quantile {quantile} loss: {loss}")


Quantile 1 loss: 17.426268223714434
Quantile 2 loss: 27.715977000191216
Quantile 3 loss: 33.48316678992623
Quantile 4 loss: 37.06400154987512
Quantile 5 loss: 40.45875252892038
Quantile 6 loss: 40.05876352364795
Quantile 7 loss: 35.35942640131153
Quantile 8 loss: 29.02211316101755
Quantile 9 loss: 19.35992468161804


In [19]:
import plotly.graph_objects as go

# Create an empty figure
fig = go.Figure()

# Plot actual values
fig.add_trace(go.Scatter(
    x=df_resampled_merged_wind_2.index,  # Assuming the index is time or some sequence
    y=y_wind,  # Actual values
    mode='lines',
    name='Actual Generation',
    line=dict(color='black', width=2)  # Style for actual values
))

# Plot predicted values for each quantile
for quantile in quantiles:
    fig.add_trace(go.Scatter(
        x=df_resampled_merged_wind_2.index,  # Assuming same x-axis for predictions
        y=df_resampled_merged_wind_2[f'generation_mw_quantile_{quantile}'],  # Predictions for each quantile
        mode='lines',
        name=f'Quantile {quantile} Prediction',
        line=dict(width=1.5)  # Style for predicted values
    ))

# Update layout for better visualization
fig.update_layout(
    title='Actual vs Predicted Wind Power Generation',
    xaxis_title='Time',
    yaxis_title='Power Generation (MW)',
    legend_title='Legend',
    hovermode='x',
    template='plotly_white'
)

# Show the plot
fig.show()


In [20]:
path = "D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/models/lgbr_model/models/i5_models/lgbr_q"
quantiles = [1, 2, 3, 4, 5, 6, 7, 8, 9]
losses = []

# Berechnung der Verluste für jedes Quantil
for i in quantiles:
    model_light = pickle.load(open(path + str(i) + ".pkl", 'rb'))
    predictions = model_light.predict(X_solar)
    loss = modified_pinball_loss(Y_solar.values, predictions, i/10)
    losses.append(loss)
    print(f"Quantile {i/10}: {loss*2779.3337282577586}")

Quantile 0.1: 5.04078413579317
Quantile 0.2: 7.035418804004097
Quantile 0.3: 9.27302369977849
Quantile 0.4: 9.79604789244977
Quantile 0.5: 9.671808948735155
Quantile 0.6: 9.881266990860778
Quantile 0.7: 9.559012269749593
Quantile 0.8: 9.159077016907492
Quantile 0.9: 4.938465481075339


In [21]:
import plotly.express as px
# Erstellen eines DataFrames für Plotly
df = pd.DataFrame({
    'Quantile': [i/10 for i in quantiles],
    'Loss': losses
})

# Plotten der Verluste mit Plotly
fig = px.line(df, x='Quantile', y='Loss', markers=True, title='Modified Pinball Loss for Different Quantiles')
fig.update_layout(
    xaxis_title='Quantile',
    yaxis_title='Modified Pinball Loss',
    template='plotly_white'
)
fig.show()

In [22]:
import pickle
import plotly.express as px
import pandas as pd

# Pfad zu den Modellen
path = "D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/models/lgbr_model/models/i5_models/lgbr_q"
quantiles = [1, 2, 3, 4, 5, 6, 7, 8, 9]
predictions_dict = {}

# Berechnung der Vorhersagen für jedes Quantil
for i in quantiles:
    model_light = pickle.load(open(path + str(i) + ".pkl", 'rb'))
    predictions = model_light.predict(X_solar)
    predictions_dict[f'Quantile {i/10}'] = predictions

# Erstellen eines DataFrames für Plotly
df = pd.DataFrame(predictions_dict)
df['Actual'] = Y_solar.values

# Plotten der Vorhersagen und der tatsächlichen Werte mit Plotly
fig = px.line(df, title='Predictions and Actual Values for Different Quantiles')
for quantile in predictions_dict.keys():
    fig.add_scatter(x=df.index, y=df[quantile], mode='lines', name=quantile)
fig.add_scatter(x=df.index, y=df['Actual'], mode='lines', name='Actual', line=dict(color='black', width=2))
fig.update_layout(
    xaxis_title='Index',
    yaxis_title='Value',
    template='plotly_white'
)
fig.show()

### create training data

In [23]:
df_solar_history = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/paul_analyse/train.csv')
df_wind_history = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/HEFTcom24/data/wind2.csv')
df_day_ahead = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/day_ahead_price.csv')
df_imbalance = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/imbalance_price.csv')
market_index = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/market_index.csv')

In [24]:
df_together = pd.merge(df_solar_history, df_wind_history, how='inner', left_on='timestamp_utc', right_on='valid_time')
df_together["Target_MW"] = df_together["Wind_MWh_credit"] + df_together["Solar_MWh_credit"]
df_together.columns

Index(['timestamp_utc', 'Mean_SolarRadiation_dwd',
       'SolarDownwardRadiation_RW_dwd_Mean_30min',
       'SolarDownwardRadiation_RW_dwd_Mean_1h',
       'SolarDownwardRadiation_dwd_Mean_Lag_30min',
       'SolarDownwardRadiation_dwd_Mean_Lag_1h',
       'SolarDownwardRadiation_dwd_Mean_Lag_24h', 'Panel_Efficiency_dwd_mean',
       'Panel_Efficiency_dwd_std', 'Panel_Temperature_dwd_mean',
       'Panel_Temperature_dwd_std', 'Std_Temperature_dwd',
       'Mean_Temperature_dwd', 'cos_hour', 'cos_day', 'solar_mw_lag_48h',
       'capacity_mwp_lag_48h', 'Target_Capacity_MWP%_lag_48h',
       'Target_Capacity_MWP%', 'Solar_MWh_credit', 'reference_time',
       'valid_time', 'RelativeHumidity_dwd', 'Temperature_dwd',
       'WindDirection_dwd', 'WindDirection:100_dwd', 'WindSpeed^3_dwd',
       'WindSpeed:100^3_dwd', 'WindSpeed_dwd', 'WindSpeed:100_dwd',
       'WindSpeed^3:100_dwd', 'RelativeHumidity_ncep', 'Temperature_ncep',
       'WindDirection_ncep', 'WindDirection:100_ncep', 'WindS

In [25]:
df_together_solar = df_together[[ 
    "Mean_SolarRadiation_dwd",
    "SolarDownwardRadiation_RW_dwd_Mean_1h",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_24h",
    "Panel_Efficiency_dwd_mean",
    "Panel_Efficiency_dwd_std",
    "Panel_Temperature_dwd_mean",
    "Panel_Temperature_dwd_std",
    "Std_Temperature_dwd",
    "Mean_Temperature_dwd",
    "cos_hour",
    "cos_day",
    "solar_mw_lag_48h",
    "capacity_mwp_lag_48h",
    "Target_Capacity_MWP%_lag_48h",
    ]]
df_together_solar.dropna(inplace=True)
mean_to_multiply = df_together_solar["capacity_mwp_lag_48h"].mean()

path = "D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/models/lgbr_model/models/i5_models/lgbr_q"
quantiles = [1, 2, 3, 4, 5, 6, 7, 8, 9]
losses = []

# Berechnung der Verluste für jedes Quantil
for i in quantiles:
    model_light = pickle.load(open(path + str(i) + ".pkl", 'rb'))
    predictions = model_light.predict(df_together_solar)
    predictions = predictions * mean_to_multiply
    df_together[f"{i}"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_28808\3827890086.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
df_together.isna().sum()

timestamp_utc                                0
Mean_SolarRadiation_dwd                      0
SolarDownwardRadiation_RW_dwd_Mean_30min     0
SolarDownwardRadiation_RW_dwd_Mean_1h        0
SolarDownwardRadiation_dwd_Mean_Lag_30min    0
SolarDownwardRadiation_dwd_Mean_Lag_1h       0
SolarDownwardRadiation_dwd_Mean_Lag_24h      0
Panel_Efficiency_dwd_mean                    0
Panel_Efficiency_dwd_std                     0
Panel_Temperature_dwd_mean                   0
Panel_Temperature_dwd_std                    0
Std_Temperature_dwd                          0
Mean_Temperature_dwd                         0
cos_hour                                     0
cos_day                                      0
solar_mw_lag_48h                             0
capacity_mwp_lag_48h                         0
Target_Capacity_MWP%_lag_48h                 0
Target_Capacity_MWP%                         0
Solar_MWh_credit                             0
reference_time                               0
valid_time   

In [27]:
df_together.columns

Index(['timestamp_utc', 'Mean_SolarRadiation_dwd',
       'SolarDownwardRadiation_RW_dwd_Mean_30min',
       'SolarDownwardRadiation_RW_dwd_Mean_1h',
       'SolarDownwardRadiation_dwd_Mean_Lag_30min',
       'SolarDownwardRadiation_dwd_Mean_Lag_1h',
       'SolarDownwardRadiation_dwd_Mean_Lag_24h', 'Panel_Efficiency_dwd_mean',
       'Panel_Efficiency_dwd_std', 'Panel_Temperature_dwd_mean',
       'Panel_Temperature_dwd_std', 'Std_Temperature_dwd',
       'Mean_Temperature_dwd', 'cos_hour', 'cos_day', 'solar_mw_lag_48h',
       'capacity_mwp_lag_48h', 'Target_Capacity_MWP%_lag_48h',
       'Target_Capacity_MWP%', 'Solar_MWh_credit', 'reference_time',
       'valid_time', 'RelativeHumidity_dwd', 'Temperature_dwd',
       'WindDirection_dwd', 'WindDirection:100_dwd', 'WindSpeed^3_dwd',
       'WindSpeed:100^3_dwd', 'WindSpeed_dwd', 'WindSpeed:100_dwd',
       'WindSpeed^3:100_dwd', 'RelativeHumidity_ncep', 'Temperature_ncep',
       'WindDirection_ncep', 'WindDirection:100_ncep', 'WindS

In [28]:
df_together["Temperature"] = df_together["Temperature_dwd"]
df_together["RelativeHumidity"] = df_together["RelativeHumidity_dwd"]
df_together["WindSpeed"] = df_together["WindSpeed_dwd"]
df_together["WindSpeed:100"] = df_together["WindSpeed:100_dwd"]

In [29]:
df_together_wind = set_up_wind_features(df_together)

In [30]:
df_together_wind1 = df_together_wind[[
    'WindSpeed:100_dwd', 'Temperature_avg', 'RelativeHumidity_avg', 'AirDensity',"UsableWindPower_opt", 'WindSpeed:100_dwd_lag1', 'WindSpeed:100_dwd_lag2', 'WindSpeed:100_dwd_lag3'
]]
df_together_wind1.dropna(inplace=True)
value_to_add = df_together_wind["PowerOutput_full"] / 2

C:\Users\paulh\AppData\Local\Temp\ipykernel_28808\3796758485.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
path = 'D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/Generation_forecast/Wind_forecast/models/gbr_quantile_0.'
quantiles = [1, 2, 3, 4, 5, 6, 7, 8, 9]
losses = []

for quantile in quantiles:
    with open(f"{path}{quantile}_res-True_calc-False.pkl", "rb") as f:
        model_wind = load_pickle1(f)
    
    if not hasattr(model_wind, '_preprocessor'):
        model_wind._preprocessor = None

    predictions_wind = model_wind.predict(df_together_wind1)
    predictions_wind = predictions_wind + value_to_add[3:]
    df_together[f"{quantile}"] = df_together[f"{quantile}"][4:] + predictions_wind

In [33]:
df_together

,timestamp_utc,Mean_SolarRadiation_dwd,SolarDownwardRadiation_RW_dwd_Mean_30min,SolarDownwardRadiation_RW_dwd_Mean_1h,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_dwd_Mean_Lag_1h,SolarDownwardRadiation_dwd_Mean_Lag_24h,Panel_Efficiency_dwd_mean,Panel_Efficiency_dwd_std,Panel_Temperature_dwd_mean,...,WindSpeed_full_avg,WindPower_full,UsableWindPower_full,PowerOutput_full,Temperature_avg,RelativeHumidity_avg,WindSpeed:100_dwd_lag1,WindSpeed:100_dwd_lag2,WindSpeed:100_dwd_lag3,UsableWindPower_opt
0,2020-09-22 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188614,0.000551,13.036279,...,5.245064,509.390097,509.390097,158.387177,16.023140,85.735146,NaN,NaN,NaN,509.390097
1,2020-09-22 00:30:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188661,0.000563,12.971021,...,5.121709,478.569212,478.569212,147.661509,16.041790,85.351560,6.374486,NaN,NaN,478.569212
2,2020-09-22 01:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188708,0.000577,12.905762,...,4.998354,449.018088,449.018088,137.377717,16.060438,84.967970,6.243333,6.374486,NaN,449.018088
3,2020-09-22 01:30:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188702,0.000595,12.914154,...,4.868452,406.265595,406.265595,122.499850,16.015215,83.303420,6.112179,6.243333,6.374486,406.265595
4,2020-09-22 02:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188696,0.000616,12.922546,...,4.738551,366.309231,366.309231,108.595035,15.969991,81.638880,5.911122,6.112179,6.243333,366.309231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50026,2023-08-25 23:30:00+00:00,0.009818,0.009818,0.004048,-0.001721,-0.003729,0.004615,0.187994,0.000419,13.896639,...,6.096540,517.393352,517.393352,161.172309,16.262043,80.340330,7.193197,7.253932,7.314668,517.393352
50027,2023-08-26 00:00:00+00:00,0.021356,0.021356,0.015587,0.009818,-0.001721,0.017088,0.188246,0.000398,13.547243,...,5.425915,352.220570,352.220570,103.692181,15.913723,84.427540,6.408854,7.193197,7.253932,352.220570
50028,2023-08-26 00:30:00+00:00,0.012875,0.012875,0.017116,0.021356,0.009818,-0.000814,0.188469,0.000386,13.237374,...,5.305044,327.673709,327.673709,95.149873,15.783813,84.867800,5.635854,6.408854,7.193197,327.673709
50029,2023-08-26 01:00:00+00:00,0.004395,0.004395,0.008635,0.012875,0.021356,-0.018716,0.188692,0.000381,12.927505,...,5.184173,304.287730,304.287730,87.011553,15.653904,85.308050,5.500908,5.635854,6.408854,304.287730


In [34]:
df_together.dropna(inplace=True)

In [35]:
df_together.timestamp_utc = pd.to_datetime(df_together.timestamp_utc)
df_day_ahead.timestamp_utc = pd.to_datetime(df_day_ahead.timestamp_utc)
df_imbalance.timestamp_utc = pd.to_datetime(df_imbalance.timestamp_utc)
market_index.timestamp_utc = pd.to_datetime(market_index.timestamp_utc)

In [36]:
df_day_ahead.dtypes

timestamp_utc        datetime64[ns, UTC]
settlement_date                   object
settlement_period                  int64
price                            float64
dtype: object

In [37]:
df_together1 = pd.merge(df_together, df_day_ahead, how='inner', left_on='timestamp_utc', right_on='timestamp_utc')
df_together2 = pd.merge(df_together1, df_imbalance, how='inner', left_on='timestamp_utc', right_on='timestamp_utc')
df_together3 = pd.merge(df_together2, market_index, how='inner', left_on='timestamp_utc', right_on='timestamp_utc')
df_together3

,timestamp_utc,Mean_SolarRadiation_dwd,SolarDownwardRadiation_RW_dwd_Mean_30min,SolarDownwardRadiation_RW_dwd_Mean_1h,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_dwd_Mean_Lag_1h,SolarDownwardRadiation_dwd_Mean_Lag_24h,Panel_Efficiency_dwd_mean,Panel_Efficiency_dwd_std,Panel_Temperature_dwd_mean,...,settlement_period_x,price_x,settlement_date_y,settlement_period_y,imbalance_price,settlement_date,settlement_period,data_provider,price_y,volume
0,2021-02-19 23:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.191314,0.000215,9.285992,...,47,32.38,2021-02-19,47,32.75000,2021-02-19,47,APXMIDP,26.46,671.75
1,2021-02-19 23:30:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.191230,0.000199,9.402743,...,48,32.38,2021-02-19,48,62.50000,2021-02-19,48,APXMIDP,29.81,905.45
2,2021-02-21 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.190715,0.000309,10.118579,...,1,17.34,2021-02-21,1,11.79015,2021-02-21,1,APXMIDP,7.67,781.05
3,2021-02-21 00:30:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.190812,0.000289,9.983511,...,2,17.34,2021-02-21,2,12.29000,2021-02-21,2,APXMIDP,9.82,694.45
4,2021-02-21 01:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.190909,0.000278,9.848444,...,3,14.25,2021-02-21,3,16.88000,2021-02-21,3,APXMIDP,10.40,847.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42280,2023-08-25 23:30:00+00:00,0.009818,0.009818,0.004048,-0.001721,-0.003729,0.004615,0.187994,0.000419,13.896639,...,2,84.41,2023-08-26,2,66.01000,2023-08-26,2,APXMIDP,83.91,1201.85
42281,2023-08-26 00:00:00+00:00,0.021356,0.021356,0.015587,0.009818,-0.001721,0.017088,0.188246,0.000398,13.547243,...,3,79.96,2023-08-26,3,66.01000,2023-08-26,3,APXMIDP,82.13,1066.95
42282,2023-08-26 00:30:00+00:00,0.012875,0.012875,0.017116,0.021356,0.009818,-0.000814,0.188469,0.000386,13.237374,...,4,79.96,2023-08-26,4,66.01000,2023-08-26,4,APXMIDP,78.76,997.60
42283,2023-08-26 01:00:00+00:00,0.004395,0.004395,0.008635,0.012875,0.021356,-0.018716,0.188692,0.000381,12.927505,...,5,73.84,2023-08-26,5,66.01000,2023-08-26,5,APXMIDP,77.47,913.25


In [178]:
df_together3.to_csv("bidding_training.csv", index=False)